In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import time

import tensorflow as tf

from context import QNetOptimizer as QNopt

In [2]:
def bell_state_local_RY(settings, wires):
    qml.Hadamard(wires[0])
    qml.CNOT(wires=wires[0:2])
        
    qml.RY(settings[0], wires=wires[0])
    qml.RY(settings[1], wires=wires[1])

def static_nlocal_prepare_nodes(n):
    return [QNopt.PrepareNode(1, [2*i, 2*i + 1], bell_state_local_RY, 2) for i in range(n)]

def local_RY_measure_nodes(n):
    meas_nodes = [QNopt.MeasureNode(2, 2, [0], QNopt.local_RY, 1)]
    meas_nodes.extend([
        QNopt.MeasureNode(2, 2, [2*i + 1, 2*i + 2], QNopt.local_RY, 2) for i in range(0,n-1)
    ])
    meas_nodes.append(QNopt.MeasureNode(2, 2, [2*n-1], QNopt.local_RY, 1))
    
    return meas_nodes

## Comparison Between Gradient Methods for n-Local Chain Optimizations

In [3]:
n = 6
prep_nodes = static_nlocal_prepare_nodes(n)
meas_nodes = local_RY_measure_nodes(n)

nlocal_ansatz = QNopt.NetworkAnsatz(prep_nodes, meas_nodes)

## Interface Comparison on Backpropagation

In [ ]:
%%time

autograd_cost = QNopt.nlocal_chain_cost_22(nlocal_ansatz, interface="autograd")

np.random.seed(1)
autograd_settings = nlocal_ansatz.rand_scenario_settings()

autograd_opt_dict = QNopt.gradient_descent(
    autograd_cost,
    autograd_settings,
    num_steps=15,
    step_size=0.8,
    sample_width = 3
)

print("max score : ", autograd_opt_dict["opt_score"])

In [4]:
%%time

nlocal_ansatz.set_interface("tf")

print(nlocal_ansatz.interface)

tf_cost = QNopt.nlocal_chain_cost_22(nlocal_ansatz, interface="tf")

np.random.seed(1)
tf_settings = nlocal_ansatz.rand_scenario_settings()


tf_opt_dict = QNopt.gradient_descent(
    tf_cost,
    tf_settings,
    num_steps=15,
    step_size=0.8,
    sample_width = 3
)

print("max score : ", tf_opt_dict["opt_score"])

tf
prepare settings :  True
measure settings :  False
iteration :  0 , score :  tf.Tensor(0.29101182584748303, shape=(), dtype=float64)


ValueError: TypeError: object of type 'numpy.float32' has no len()
Traceback (most recent call last):

  File "/Users/brian/opt/anaconda3/envs/pennylane-nonlocality-dev/lib/python3.8/site-packages/autograd/numpy/numpy_boxes.py", line 21, in __len__
    def __len__(self): return len(self._value)

TypeError: object of type 'numpy.float32' has no len()

